In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv',nrows=10_000_000)
df.head(5)

In [ ]:
for col in df.columns:
    print(col,"..................")
    print(df[f'{col}'].value_counts(),'\n')

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['manhattan_distance']=((df.dropoff_latitude - df.pickup_latitude)+ (df.dropoff_longitude - df.pickup_longitude)).abs()
df['Difference_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
df['Difference_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

In [ ]:
df.head(5)

In [ ]:
ls1=df['pickup_datetime'].tolist()
for i in range(len(ls1)):
    ls1[i]=ls1[i][11:-7:]
df['pickuptime']=ls1    

ls1=df['pickuptime'].tolist()
for i in range(len(ls1)):
    z=ls1[i].split(':')
    ls1[i]=int(z[0])*100+int(z[1])
df['pickuptime']=ls1

In [ ]:
df.head(5)

In [ ]:
import numpy as np
ls1=df['pickup_datetime'].tolist()
for i in range(len(ls1)):
    ls1[i]=ls1[i][:-4]
    ls1[i]=pd.Timestamp(ls1[i])
    ls1[i]=ls1[i].weekday()
    
from sklearn.preprocessing import OneHotEncoder

encoder=OneHotEncoder()
encodedls1=encoder.fit_transform(np.array(ls1).reshape(len(ls1),1))


In [ ]:
encodedls1=encodedls1.toarray()
print(len(encodedls1[0]))
days=['mon','tue','wed','thr','fri','sat','sun']
for i in range(len(days)):
     df[days[i]]=encodedls1[:,i]

In [ ]:
df.head(5)

In [ ]:
R = 6373.0
lat1 =np.asarray(np.radians(df['pickup_latitude']))
lon1 = np.asarray(np.radians(df['pickup_longitude']))
lat2 = np.asarray(np.radians(df['dropoff_latitude']))
lon2 = np.asarray(np.radians(df['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1
ls1=[] 
a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/ 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
distance = R * c

    
df['Distance']=np.asarray(distance)*0.621

In [ ]:
R = 6373.0
lat1 =np.asarray(np.radians(df['pickup_latitude']))
lon1 = np.asarray(np.radians(df['pickup_longitude']))
lat2 = np.asarray(np.radians(df['dropoff_latitude']))
lon2 = np.asarray(np.radians(df['dropoff_longitude']))

lat3=np.zeros(len(df))+np.radians(40.6413111)
lon3=np.zeros(len(df))+np.radians(-73.7781391)
dlon_pickup = lon3 - lon1
dlat_pickup = lat3 - lat1
d_lon_dropoff=lon3 -lon2
d_lat_dropoff=lat3-lat2
a1 = np.sin(dlat_pickup/2)**2 + np.cos(lat1) * np.cos(lat3) * np.sin(dlon_pickup/ 2)**2
c1 = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1 - a1))
distance1 = R * c1
df['Pickup_Distance_airport']=np.asarray(distance1)*0.621

a2=np.sin(d_lat_dropoff/2)**2 + np.cos(lat2) * np.cos(lat3) * np.sin(d_lon_dropoff/ 2)**2
c2 = 2 * np.arctan2(np.sqrt(a2), np.sqrt(1 - a2))
distance2 = R * c2

    
df['Dropoff_Distance_airport']=np.asarray(distance2)*0.621

In [ ]:
df['Distance']=np.round(df['Distance'],2)
df['Pickup_Distance_airport']=np.round(df['Pickup_Distance_airport'],2)
df['Dropoff_Distance_airport']=np.round(df['Dropoff_Distance_airport'],2)

In [ ]:
df.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)

In [ ]:
df['Difference_longitude']=np.abs(df['Difference_longitude']-np.mean(df['Difference_longitude']))
df['Difference_longitude']=df['Difference_longitude']/np.var(df['Difference_longitude'])

In [ ]:
df['Difference_latitude']=np.abs(df['Difference_latitude']-np.mean(df['Difference_latitude']))
df['Difference_latitude']=df['Difference_latitude']/np.var(df['Difference_latitude'])

In [ ]:
df.head(5)

In [ ]:
test_data=pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')
test_data.dropna(inplace=True)
test_data.head(5)

In [ ]:


test_data['manhattan_distance']=((test_data.dropoff_latitude - test_data.pickup_latitude)+ (test_data.dropoff_longitude - test_data.pickup_longitude)).abs()
test_data['Difference_longitude'] = (test_data.dropoff_longitude - test_data.pickup_longitude).abs()
test_data['Difference_latitude'] = (test_data.dropoff_latitude - test_data.pickup_latitude).abs()

ls1=test_data['pickup_datetime'].tolist()
for i in range(len(ls1)):
    ls1[i]=ls1[i][11:-7:]
test_data['pickuptime']=ls1  

ls1=test_data['pickuptime'].tolist()
for i in range(len(ls1)):
    z=ls1[i].split(':')
    ls1[i]=int(z[0])*100+int(z[1])
test_data['pickuptime']=ls1

ls2=test_data['pickup_datetime'].tolist()
for i in range(len(ls1)):
    ls2[i]=ls2[i][:-4]
    ls2[i]=pd.Timestamp(ls1[i])
    ls2[i]=ls2[i].weekday()

encodedls2=encoder.transform(np.array(ls2).reshape(len(ls2),1))
encodedls2=encodedls2.toarray()
    
for i in range(len(days)):
     test_data[days[i]]=encodedls2[:,i]
        
lat1 =np.asarray(np.radians(test_data['pickup_latitude']))
lon1 = np.asarray(np.radians(test_data['pickup_longitude']))
lat2 = np.asarray(np.radians(test_data['dropoff_latitude']))
lon2 = np.asarray(np.radians(test_data['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1
 
a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/ 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
distance = R * c
test_data['Distance']=np.asarray(distance)*0.621


In [ ]:
R = 6373.0

lat1 =np.asarray(np.radians(test_data['pickup_latitude']))
lon1 = np.asarray(np.radians(test_data['pickup_longitude']))
lat2 = np.asarray(np.radians(test_data['dropoff_latitude']))
lon2 = np.asarray(np.radians(test_data['dropoff_longitude']))

lat3=np.zeros(len(test_data))+np.radians(40.6413111)
lon3=np.zeros(len(test_data))+np.radians(-73.7781391)
dlon_pickup = lon3 - lon1
dlat_pickup = lat3 - lat1
d_lon_dropoff=lon3 -lon2
d_lat_dropoff=lat3-lat2
a1 = np.sin(dlat_pickup/2)**2 + np.cos(lat1) * np.cos(lat3) * np.sin(dlon_pickup/ 2)**2
c1 = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1 - a1))
distance1 = R * c1
test_data['Pickup_Distance_airport']=np.asarray(distance1)*0.621

a2=np.sin(d_lat_dropoff/2)**2 + np.cos(lat2) * np.cos(lat3) * np.sin(d_lon_dropoff/ 2)**2
c2 = 2 * np.arctan2(np.sqrt(a2), np.sqrt(1 - a2))
distance2 = R * c2

    
test_data['Dropoff_Distance_airport']=np.asarray(distance2)*0.621

In [ ]:
test_data['Distance']=np.round(test_data['Distance'],2)
test_data['Pickup_Distance_airport']=np.round(test_data['Pickup_Distance_airport'],2)
test_data['Dropoff_Distance_airport']=np.round(test_data['Dropoff_Distance_airport'],2)

test_data.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)

test_data['Difference_longitude']=np.abs(test_data['Difference_longitude']-np.mean(test_data['Difference_longitude']))
test_data['Difference_longitude']=test_data['Difference_longitude']/np.var(test_data['Difference_longitude'])

test_data['Difference_latitude']=np.abs(test_data['Difference_latitude']-np.mean(test_data['Difference_latitude']))
test_data['Difference_latitude']=test_data['Difference_latitude']/np.var(test_data['Difference_latitude'])


In [ ]:

test_data.head(5)

In [ ]:
df.head(5)

In [ ]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test=train_test_split(df.drop(['key','fare_amount','pickup_datetime'],axis=1),df['fare_amount'],test_size=0.01,random_state=80)

df=[1,2,3,4,5]

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression(normalize=True)
lr.fit(X_train,y_train)
print(lr.score(X_test,y_test))

In [ ]:
X_test.info()

In [ ]:
pred=np.round(lr.predict(test_data.drop(['key','pickup_datetime'],axis=1)),2)

In [ ]:
pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv').head()

Submission=pd.DataFrame(data=pred,columns=['fare_amount'])
Submission['key']=test_data['key']
Submission=Submission[['key','fare_amount']]

Submission.set_index('key',inplace=True)

Submission.head(5)

In [ ]:
Submission.to_csv('Submission.csv')